# 后训练技术

## 后训练（Post-training）
后训练是在 **大模型预训练后获得的基础模型** 上进行的下一步训练。

其核心是利用细心筛选过的数据再次训练基础模型，使其能够对指令作出响应或与用户进行交流，成为升级为指令模型或对话模型。

针对升级后的模型，我们甚至可以展开更进一步的训练，以调整模型行为或增强特定能力，从而获得一个定制化的模型，使其可以 **专精于某一特定领域，具备特定的行为模式**。

![introduction1.png](images/introduction1.png)

## 预训练（Pre-training）

预训练通常为无监督学习。

通过大规模无标注语料训练成可以根据已见标记预测下一个标记的模型。

![introduction3.png](images/introduction3.png)


## 后训练方法

### 监督微调（SFT）

监督微调是最简单且最流行的后训练方法，使用包含标注的提示-响应对数据集进行模型训练，属于监督学习/模仿学习的范畴。

其中，**提示**是给模型的指令，**响应**是模型应有的理想回答。

优点：训练过程仅需1,000至10亿个标记，远少于预训练规模。

缺点：仅对响应标记进行训练，而不涉及提示标记。

![introduction4.png](images/introduction4.png)

### 直接偏好优化（DPO）

直接偏好优化使用包含提示及其对应优质/劣质响应的数据集进行模型训练。

针对任一提示，使模型生成多个响应并筛选出优质与劣质样本。通过构造性损失函数，使模型 **趋近优质响应而远离劣质响应**。

优点：仅需1,000至10亿个标记。

缺点：采用更复杂的损失函数。

![introduction5.png](images/introduction5.png)

### 在线强化学习（Online RL）
在线强化学习只需准备提示集和奖励函数。 

通过提示让语言模型生成响应，再通过奖励函数对该响应进行评分，最后利用该信号更新模型。

通常需要1,000至1,000万（或更多）个提示，目标是**通过模型自身生成的响应来最大化奖励值**。

![introduction6.png](images/introduction6.png)

## 后训练三要素
1. 数据与算法协同设计：每种后训练方法所需的数据结构略有不同，良好的协同设计对后训练成效至关重要。

2. 可靠高效的算法库。

   - HuggingFace TRL、Open RLHF、veRL和Nemo RL等。

3. 合适的评估体系：通过完善的评估方案追踪模型在后训练前后的表现，确保模型性能持续优良。

   - 对话机器人竞技场：基于人类偏好的聊天评估。
   - 替代人类评判的LLM评估：AlpacaEval、MT Bench、Arena Hard。
   - 指令模型静态基准：LiveCodeBench（热门代码基准）、AIME 2024/2025（高难度数学评估）。
   - 知识与推理数据集：GPQA、MMLU Pro。
   - 指令遵循评估：IFEval。
   - 函数调用与智能体评估：BFCL、NexusBench、TauBench、ToolSandbox（后两者专注多工具使用场景）。

## 后训练场景

**并非所有大模型都需要进行后训练。**

- 若仅需模型遵循少量指令（如回避敏感话题或禁止公司间比较），通过**提示工程**即可实现，但该方法虽简单却不够稳定。

- 如需查询实时数据库，**检索增强生成**或基于搜索的方法可能更适用。

- 创建领域专用模型（如医疗或网络安全语言模型）时，通常需要**持续预训练结合标准后训练**，先让模型学习领域知识（至少需10亿标记），再学习用户交互。

- 当需要严格遵循20条以上指令，或提升特定能力（如构建强SQL模型、函数调用模型或推理模型）时，后训练最能发挥价值——它能可靠改变模型行为并提升目标能力，但若实施不当可能导致其他未训练能力退化。